In [15]:
from collections import Counter

from rlcache.backend import storage_from_config
from rlcache.cache_constants import CacheStatus
from rlcache.cache_manager import CacheManager
from rlcache.strategies.caching_strategies import caching_strategy_from_config
from rlcache.strategies.caching_strategies.rl_caching_strategy import CachingStrategyRLConverter
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

CONFIG = {
  "experiment_name": "RLCaching strategy only",
  "cache_backend_settings": {
    "type": "inmemory",
    "capacity": 5000
  },
  "database_backend_settings": {
    "type": "inmemory",
    "capacity": 1000000000
  },
  "cache_manager_settings": {
    "caching_strategy_settings": {
      "type": "rl_driven"
    },
    "ttl_strategy_settings": {
      "type": "fixed",
      "ttl": 500
    },
    "eviction_strategy_settings": {
      "type": "lru"
    }
  }
}
DATABASE_BACKEND = storage_from_config(CONFIG['database_backend_settings'])
CACHE_BACKEND = storage_from_config(CONFIG['cache_backend_settings'])
CACHE_MANAGER = CacheManager(config=CONFIG['cache_manager_settings'],
                             cache=CACHE_BACKEND,
                             backend=DATABASE_BACKEND)
REQUESTS_COUNTER = Counter()

KeyError: 'num_fields'

In [92]:
caching_config = {"type": "rl_driven",
                  "num_fields": 10,
                  'max_capacity': 5000,
                  "agent_config": {
                      "type": "random-agent",
                      "network_spec": {},
                      "preprocessing_spec": [],
                      "execution_spec": {
                          "seed": 15,
                          "gpu_spec": {
                              "gpus_enabled": True
                          }
                      },
                  },
                  }
    
caching_strategy = caching_strategy_from_config(caching_config)

19-04-02 18:49:36:INFO:Parsed state space definition: Dict([('capacity', "Intbox(() <class 'numpy.int32'> )"), ('key', "Textbox((1, 1) <class 'numpy.str_'> )"), ('values', "Textbox((10, 1) <class 'numpy.str_'> )")])
19-04-02 18:49:36:INFO:Parsed action space definition: Boolbox(() <class 'numpy.bool_'> )
19-04-02 18:49:36:INFO:No preprocessing required.
19-04-02 18:49:36:INFO:Execution spec is: {'seed': 15, 'gpu_spec': {'gpus_enabled': True}, 'mode': 'single', 'distributed_spec': None, 'disable_monitoring': False, 'device_strategy': 'default', 'default_device': None, 'device_map': {}, 'session_config': {'type': 'monitored-training-session', 'allow_soft_placement': True, 'log_device_placement': False}, 'enable_profiler': False, 'profiler_frequency': 1000, 'enable_timeline': False, 'timeline_frequency': 1}
19-04-02 18:49:36:WARNING:WARNING: max_usable_gpus is 1 but only 0 gpus are locally visible. Using all available GPUs.
19-04-02 18:49:36:INFO:GPU strategy initialized with GPUs enabled

/home/samialab/anaconda3/envs/rlcache/lib/python3.6/site-packages/rlgraph/graphs/tensorflow_executor.py:757: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "Using all available GPUs.".format(self.max_usable_gpus, len(gpu_names)))


In [100]:
dummy_key = 'user6868534811834787757'
dummy_values =  {'field1': '2Ki"-2$>z\'087Vq.7d"+:)Ec1Iw7/`0Y5(#<%Fm"$:4%h6G\'\'0 !M=#"6=;j"C+86p;+>9Vy?L546z9@1=Ze3/`?^y0Vo7/|94n/',
                 'field0': '(Ei#Lw*4404n0>z22<"]5)8*$/b;Q345~2Aq\':8!6:=Q7-Z1((l76j;%>)Du?A7,#f/1t*&d=Ke#:";@#,]o,A}.!b9Ic;(>17r;',
                 'field7': '=/>0Pu:M59P)$(>$G9);j"Ik(&,(44=;>1*,7Ai&Y+(#6)Ri/6"996)R#8!b?Ms\'#4-2.= |-(v+24\'Ge3E!35x\'+4![{#Tu(%"9', 
                 'field6': ',(d&\\!"?>5),3%(%Xw?D-$(n4Rw\'%./:,!Ls:7$(Ay9949@3(.f"_m7Sq/B=4X{;:.(?b"L%==8<[i802 62 ^q"?t6%09D5;E\' ', 
                 'field9': '(344S)-_%4:"\'_a6&. ?n*Ks:.p.>$6+t\' 8)A+-Ho)((9Ig,,4,?h-\'b/@m/>x&E74]{6A+1> 09~5[u7F%>4`+?64J#=485Ym2',
                 'field8': '&Ls.Z3,H90S!(Gc7?"#F)-Q97&v\'-"/ t*@\x7f>-z/Qs >&,,2=X1=M9&=v#9*#)|7^c [!)\\7&\'j3!n3$t?Ay\'U!4Hy%-4%[7>L)%',
                 'field3': "/.`;%h>&j/K3-G{7V%1Uy>'&$\\-4E%8(t+F= ? )-d%\\\x7f6R)9H)$A')I\x7f+<23Di/I7>Yc>W{$B5)/>3*z:Je 2`'5r+Uq>^?&Rs#", 
                 'field2': '7B; ,|,#|>:40Yc*^y6"*51p\'1z7[#-X=-Su$Pc"Y\x7f\'.d;Ma8\\-- ~7+,9+t$Y{/Mg"3l.@u1J!$?"(Gy2Fa:0d.Si92,4M32Cu#',
                 'field5': '+*6 \\g"9d/:: !:!$26 8 A+60l2 v 4n#P\'7^)!J72021\'t##r>;,8De;X--2b2*z$)* >f%@c3<85Ay37z9Aq/R5*O7) b3Ku>', 
                 'field4': '(Ca2<:*Hg7O/26l>-27;0<T)8Wm?.6+2*1^}>+ 0No;Ss2Dg>3h2]q!+*1Ec1Xg.]3&3~.?0\'<.$Ru3)j3,.?Q1-L!#Py\'&,"$v&'}

# TODO the workload generator generates stupid values
caching_strategy.should_cache(dummy_key, dummy_values, CacheStatus.New)

IndexError: too many indices for array

In [52]:
converter_test = CachingStrategyRLConverter()

In [56]:
converter_test.system_to_agent_state(key=dummy_key, 
                                     value=dummy_values, 
                                     status=CacheStatus.New)

array('user6868534811834787757', dtype='<U23')

In [106]:

caching_strategy.agent.state_space.sample()

{'capacity': array(1173, dtype=int32),
 'key': array([['N']], dtype='<U1'),
 'values': array([['N'],
        ['N'],
        ['N'],
        ['N'],
        ['N'],
        ['N'],
        ['N'],
        ['N'],
        ['N'],
        ['N']], dtype='<U1')}